In [ ]:
import openslide
import xml.etree.ElementTree as ET
import glob
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tqdm
import albumentations as A
import torch
from albumentations.pytorch.transforms import ToTensorV2
import Task1_handling
import pandas as pd
import seaborn as sns

In [ ]:
base_path='/mnt/dataset/*.svs'
extractor=Task1_handling.mask_extractor(base_path)

In [ ]:
def get_patches(extractor,i,w_patch,h_patch,ratio_threshold,mask_size=1,org_level=2,overlap=2):
    img=extractor.obj_slide_list[i]
    
    LEVEL_EXTRACTION=2
    if float(img.properties['aperio.MPP']) > 0.3:
        scale_factor = 1
    else:
        scale_factor = 2
    
    #print('scale_factor:{}'.format(scale_factor))
    w_4,h_4=img.level_dimensions[LEVEL_EXTRACTION]
    w_0,h_0=img.level_dimensions[0]
    
    img_np=np.array(img.read_region((0,0),LEVEL_EXTRACTION,(w_4,h_4)))[:,:,0:3]
    #print('img_np size:{}'.format(img_np.shape))
    img_np=cv2.resize(img_np,(int(w_4/scale_factor),int(h_4/scale_factor)))
    
    #print('img_np size after resize:{}'.format(img_np.shape))
    
    img_gray=cv2.cvtColor(img_np,cv2.COLOR_RGB2GRAY)

    pad_h=h_patch-img_gray.shape[0]%h_patch
    pad_w=w_patch-img_gray.shape[1]%w_patch


    img_gray=np.pad(img_gray,((0,pad_h),(0,pad_w)),'constant', constant_values=(255,255))
    #print('size after padding:{}'.format(img_gray.shape))
    _,img_gray=cv2.threshold(img_gray,235,255,cv2.THRESH_BINARY_INV)
    
    n_patch_h=img_gray.shape[0]//h_patch
    n_patch_w=img_gray.shape[1]//w_patch


    img_gray=img_gray.reshape(
        n_patch_h,
        h_patch,
        n_patch_w,
        w_patch 
        )

    img_gray=img_gray.transpose((0,2,1,3))
    
    
    
    ratio=(img_gray.reshape(img_gray.shape[0],img_gray.shape[1],-1)==255).sum(-1)/(h_patch*w_patch)
    ratio_index=np.where(ratio>=ratio_threshold,True,False)
    

    org_scale=4**(2-org_level)
    org_img_list=[]


    layer3,layer5=extractor.mask_extraction(i)
    layer3_list=[]
    layer5_list=[]
    #fig,axes=plt.subplots (n_patch_h,n_patch_w)

    for i in range(overlap*n_patch_h):
        for j in range(overlap*n_patch_w):
            if ratio_index[i//overlap][j//overlap]==True:
                #print(i,j)
                #print('ratio_index:{}'.format(ratio_index[i][j]))
                index_dict = {"x_start":}
                
                x_start=(j*w_patch)//overlap
                x_end=(j*w_patch)//overlap+w_patch
                
                y_start=(i*h_patch)//overlap
                y_end=(i*h_patch)//overlap+h_patch
                
                org_img=np.array(img.read_region((x_start*16*scale_factor,y_start*16*scale_factor),org_level,(w_patch*scale_factor*org_scale,h_patch*scale_factor*org_scale)))[:,:,0:3]
                org_img=cv2.resize(org_img,(w_patch*org_scale,h_patch*org_scale)) #org_img의 사이즈를 mpp (scale_factor)로 보정
                #mask가 layer2 크기로 설정되어 있어 만약 level이 2가 아니면 mask보다 org_img 크기가 더 커짐.
                
                layer3_patch=layer3[y_start*mask_size:y_end*mask_size,x_start*mask_size:x_end*mask_size]
                #print('layer3 location: {}:{},{}:{}'.format(i*h_patch*org_scale,(i+1)*h_patch*org_scale,j*w_patch*org_scale,(j+1)*w_patch*org_scale))
                layer5_patch=layer5[y_start*mask_size:y_end*mask_size,x_start*mask_size:x_end*mask_size]
                #print('region:{},{},scale:{},{}'.format(j*w_patch*org_scale,i*h_patch*org_scale,w_patch*org_scale,h_patch*org_scale))
                #axes[i][j].imshow(org_img) ##org_img 시각화
                #axes[i][j].imshow(layer3_patch,'gray') ##layer3시각화
                if (layer3_patch.shape)==(h_patch,w_patch):
                    org_img_list.append(org_img)
                    layer3_list.append(layer3_patch)
                    layer5_list.append(layer5_patch)
                else:
                    ratio_index[i//overlap][j//overlap]=False
            #else:
                #axes[i,j].imshow(np.zeros((w_patch,h_patch)))  
    
    if len(org_img_list)>0:
        org_img_np=np.stack(org_img_list,axis=0)
        layer3_np=np.stack(layer3_list,axis=0)
        layer5_np=np.stack(layer5_list,axis=0)
        
        print('shape of org_img:{}, shape of layer_3:{}, shape of layer_5:{}'.format(org_img_np.shape,layer3_np.shape,layer5_np.shape))
        
        return org_img_np,layer3_np,layer5_np,1
    else:
        return 0,0,0,0

    


In [ ]:
path_list=extractor.base_path_list
path_list[30]

In [ ]:
path_list=np.delete(path_list,[37,45,41,30])

In [ ]:
org_img_dataset=[]
layer3_dataset=[]
layer5_dataset=[]

for index,file_path in enumerate(path_list[:-5]):
    org_img_batch, layer3_batch, layer5_batch , info= get_patches(extractor,index,256,256,0.8)
    if info==1:
        org_img_dataset.append(org_img_batch)
        layer3_dataset.append(layer3_batch)
        layer5_dataset.append(layer5_batch)


In [ ]:
org_img_dataset=np.concatenate(org_img_dataset,axis=0)
layer3_dataset=np.concatenate(layer3_dataset,axis=0)
layer5_dataset=np.concatenate(layer5_dataset,axis=0)

In [ ]:
org_img_dataset.shape
org_img_dataset=np.array(org_img_dataset,np.int32)

In [ ]:
import argparse
import numpy as np
from PIL import Image
from scipy.linalg import eigh


def normalizeStaining(img, saveFile=None, Io=240, alpha=1, beta=0.15):
    ''' Normalize staining appearence of H&E stained images
    
    Example use:
        see test.py
        
    Input:
        I: RGB input image
        Io: (optional) transmitted light intensity
        
    Output:
        Inorm: normalized image
        H: hematoxylin image
        E: eosin image
    
    Reference: 
        A method for normalizing histology slides for quantitative analysis. M.
        Macenko et al., ISBI 2009
    '''
             
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
        
    maxCRef = np.array([1.9705, 1.0308])
    
    # define height and width of image
    h, w, c = img.shape
    
    # reshape image
    img = img.reshape((-1,3))

    # calculate optical density
    OD = -np.log((img.astype(np.float)+1)/Io)
    
    # remove transparent pixels
    ODhat = OD[~np.any(OD<beta, axis=1)]
        
    # compute eigenvectors
    cov=np.cov(ODhat.T)
    if np.isnan(cov).sum()!=0:
        return img.reshape(256,256,3)
    eigvals, eigvecs = eigh(cov)
    
    #eigvecs *= -1
    
    #project on the plane spanned by the eigenvectors corresponding to the two 
    # largest eigenvalues    
    That = ODhat.dot(eigvecs[:,1:3])
    
    phi = np.arctan2(That[:,1],That[:,0])
    
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100-alpha)
    
    vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
    vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)
    
    # a heuristic to make the vector corresponding to hematoxylin first and the 
    # one corresponding to eosin second
    if vMin[0] > vMax[0]:
        HE = np.array((vMin[:,0], vMax[:,0])).T
    else:
        HE = np.array((vMax[:,0], vMin[:,0])).T
    
    # rows correspond to channels (RGB), columns to OD values
    Y = np.reshape(OD, (-1, 3)).T
    
    # determine concentrations of the individual stains
    C = np.linalg.lstsq(HE,Y, rcond=None)[0]
    
    # normalize stain concentrations
    maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
    tmp = np.divide(maxC,maxCRef)
    C2 = np.divide(C,tmp[:, np.newaxis])
    
    # recreate the image using reference mixing matrix
    Inorm = np.multiply(Io, np.exp(-HERef.dot(C2)))
    Inorm[Inorm>255] = 254
    Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)  

    return Inorm

In [ ]:
Inorm_img_dataset=[]
for i in range(len(org_img_dataset)):
    Inorm=normalizeStaining(org_img_dataset[i])
    Inorm_img_dataset.append(Inorm)

In [ ]:
Inorm_img_dataset=np.stack(Inorm_img_dataset,axis=0)
Inorm_img_dataset.shape

In [ ]:
#sort key
key=layer3_dataset.reshape(len(org_img_dataset),-1).sum(1)/(256*256)
sns.distplot(key)
plt.show()

In [ ]:
#sort key
key=layer5_dataset.reshape(len(org_img_dataset),-1).sum(1)/(256*256)
sns.distplot(key)
plt.show()